## Análisis de ventas

#### Importar librerías necesarias para empezar

In [ ]:
import pandas as pd
import os

### Objetivo 1: Poner los 12 csv en un mismo df

In [ ]:
df = pd.read_csv('./pandas_projects/SalesAnalysis/Sales_Data/Sales_April_2019.csv')
files = [file for file in os.listdir('./pandas_projects/SalesAnalysis/Sales_Data')]
all_months_data = pd.DataFrame()

for file in files:
    df = pd.read_csv('./pandas_projects/SalesAnalysis/Sales_Data/'+file)
    all_months_data = pd.concat([all_months_data, df])

all_months_data.to_csv('all_data.csv', index=False)

Leer el 'all_data' nuevo completo

In [ ]:
all_data = pd.read_csv('all_data.csv')
all_data.head()

Borrar filas y columnas que tengan NaN

In [ ]:
nan_df = all_data[all_data.isna().any(axis=1)]
nan_df.head()
all_data =all_data.dropna(how='all')
all_data.head()

In [ ]:
all_data.head

Encontrar los "Or" y borrarlos

In [ ]:
all_data = all_data[all_data['Order Date'].str[0:2] !='Or']
all_data.head()

Convertir columnas a su formato correcto usando la funcion pd.to_formatoquequieras

In [ ]:
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'])
all_data['Price Each'] = pd.to_numeric(all_data['Price Each'])
all_data.head()

Augment data with aditional columns

### Objetivo 2: Crear columna de mes

In [ ]:
all_data['Month'] = all_data['Order Date'].str[0:2]
all_data['Month'] = all_data['Month'].astype('int32')
all_data.head()


### Objetivo 3: Crear columna de ventas

In [ ]:
all_data['Sales'] = all_data['Quantity Ordered'] * all_data['Price Each']
all_data.head()

### Objetivo 4: Agregar columna de ciudad

In [ ]:
# Usar método .apply()
def get_city(address):
    return address.split(',')[1]
def get_state(address):
    return address.split(',')[2].split(' ')[1]
all_data['City'] = all_data['Purchase Address'].apply(lambda x: f"{get_city(x)} ({get_state(x)})")
all_data.head()


#### Pregunta 1: Cuál fue el mejor mes de ventas?

In [ ]:
results = all_data.groupby('Month').sum()

In [ ]:
import matplotlib.pyplot as plt
months = range(1,13)
plt.bar(months, results['Sales'])
plt.xticks(months)
plt.ylabel('Sales in USD ($)')
plt.xlabel('Month number')
plt.show()

#### Pregunta 2: Cuál fue la ciudad donde se realizaron más ventas?

In [ ]:
results = all_data.groupby('City').sum()
results

In [ ]:
import matplotlib.pyplot as plt
cities = [city for city, df in all_data.groupby(['City'])]
plt.bar(cities, results['Sales'])
plt.xticks(cities, rotation='vertical', size=9)
plt.ylabel('Sales in USD ($)', size=9)
plt.xlabel('City name')
plt.show()

#### Pregunta 3: En qué horario se maximizaron la cantidad de ventas?

In [ ]:
all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])


In [ ]:
all_data['Hour'] = all_data['Order Date'].dt.hour
all_data['Minute'] = all_data['Order Date'].dt.minute
all_data['Count'] = 1
all_data.head()

In [ ]:
hours = [hour for hour, df in all_data.groupby('Hour')]
plt.plot(hours, all_data.groupby(['Hour']).count())
plt.xticks(hours)
plt.xlabel('Hora')
plt.ylabel('Numero de ventas')
plt.grid()
plt.show()
#Alrededor de las 11 de la mañana y de las 19 de la tarde son las mejores horas para publicidad

#### Pregunta 4: Cuáles productos se venden más seguidos juntos?

In [ ]:
df = all_data[all_data['Order ID'].duplicated(keep=False)]

df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))
df = df[['Order ID', 'Grouped']].drop_duplicates()
df.head(100)

In [ ]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in df['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

for key, value in count.most_common(10):
    print(key,value)

#### Pregunta 5: Qué producto se vendió más? Por qué pensás que se vendió más?

In [ ]:
all_data.head()

In [ ]:
product_group = all_data.groupby('Product')
quantity_ordered = product_group.sum()['Quantity Ordered']

products = [product for product, df in product_group]

plt.bar(products, quantity_ordered)
plt.ylabel('Cantidad')
plt.xlabel('Producto')
plt.xticks(products, rotation ='vertical', size=8)
plt.show()

In [ ]:
prices = all_data.groupby('Product').mean()['Price Each']
fig,ax1 =plt.subplots()
ax2 = ax1.twinx()
ax1.bar(products, quantity_ordered, color='g')
ax2.plot(products, prices, 'b-')

ax1.set_xlabel('Producto')
ax1.set_ylabel('Cantidad', color ='g')
ax2.set_ylabel('Precio', color='b')
ax1.set_xticklabels(products, rotation='vertical', size=8)
plt.show()

In [ ]:
# Gracias por llegar al final de mi código :)
# Leandro Cabral Santana